# Convert taxi zone shapefile to parquet to load into athena
- GTP 1-3-2026
- source: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
- save locally into data/taxi_zones

In [1]:
import geopandas as gpd
import boto3

ModuleNotFoundError: No module named 'geopandas'

In [3]:
# Test connection
s3 = boto3.client('s3')

# List your S3 buckets
response = s3.list_buckets()
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  amazon-sagemaker-296646497193-us-east-2-53wwjlkhfv8ox5
  geoff-analytics-nyc-tlc-data
  gtp-tlc-data
  tlc-taxi-zones


In [6]:
# 1. Read and convert shapefile
tlc_gdf = gpd.read_file('../data/taxi_zones/taxi_zones.shp')
tlc_gdf['geometry_wkt'] = tlc_gdf['geometry'].apply(lambda x: x.wkt)

# 2. Save as Parquet
tlc_gdf.to_parquet('../data/taxi_zones/taxi_zones.parquet', index=False)

ImportError: Missing optional dependency 'pyarrow.parquet'. pyarrow is required for Parquet support.  "
        "Use pip or conda to install pyarrow.parquet.

In [ ]:
# 3. Upload to S3
s3 = boto3.client('s3')
s3.upload_file('data/taxi_zones/taxi_zones.parquet', 'tlc-taxi-zones', 'taxi_zones.parquet')

print("Shapefile loaded to S3! Now create the Athena table using the SQL above.")